### Open Street Service Notebook

Coded up with help of Chatgpt


### How to encode the info

How are we going to use this data? 

searching for places within the 1km radius of the house

Need to be able to get the coords of the place so can work out distance + route(?) by car to both train station and CBD + 
- Train stations 
- CBD
- parks
- hosptials
- supermarkets
- schools
- shopping districts



- get a rough estimate for the amount of each within the 1km radius (just a count)
- Also only print off/ Store the information of the places with either the coords or the address... 


In [1]:
# Melbourne CBD Coords (LAT, LONG)

coords_cbd = [-37.8124, 144.9623]
import csv

# Build Functions

1.the get_coordinates() outputs the coordinates for every input of address

2.the find_nearby_locations outputs the nearby(1km) amentities for every input of address coordinates

In [2]:
from geopy.geocoders import Nominatim
import requests
import folium

def get_coordinates(address):
    geolocator = Nominatim(user_agent="myGeocoder")
    location = geolocator.geocode(address)
    return (location.latitude, location.longitude)


def find_nearby_locations(lat, lon, location_type):
    tags = {
        "schools": '["amenity"="school"]',
        "parks": '["leisure"="park"]',
        "supermarkets": '["shop"="supermarket"]',
        "shopping_districts": '["shop"="mall"]',
        "train_stations": '["railway"="station"]',
        "hospitals": '["amenity"="hospital"]'
    }
    
    query = f"""
    [out:json];
    (
      node{tags[location_type]}(around:1000,{lat},{lon});
      way{tags[location_type]}(around:1000,{lat},{lon});
      relation{tags[location_type]}(around:1000,{lat},{lon});
    );
    out body;
    """
    response = requests.get('http://overpass-api.de/api/interpreter', params={'data': query})
    data = response.json()
    return data['elements']


### Process each Location

Note that has the assumption that the address of the places found are within the suburb and postcode 

This shows amenities address as much as it can

In [3]:
import re 

def process_locations(locations, address):
    
    pattern = r",\s*([A-Za-z\s]+)\s*VIC\s*(\d{4})"
    match = re.search(pattern, address)
    
    if match:
        suburb = match.group(1).strip()  # Suburb (e.g., Coburg)
        postcode = match.group(2).strip()  # Postcode (e.g., 3058)
    
    # Dictionary to store valid locations
    valid_locations = []
    
    for loc in locations:
        
        if 'tags' in loc:
            name = loc['tags'].get('name')
            location_lat = loc.get('lat')
            location_lon = loc.get('lon')

            # Get the address information from tags if available
            
            street_number = loc['tags'].get('addr:housenumber')
            street = loc['tags'].get('addr:street')
            postcode = loc['tags'].get('addr:postcode', postcode)
            suburb = loc['tags'].get('addr:suburb', suburb)
            # Construct the address
            address = f"{street_number} {street}, {suburb}, {postcode}" if street_number and street else None
            
            # Filter locations with either coordinates or address
            if (location_lat and location_lon) or address:
                valid_locations.append({
                    'name': name or "Unnamed Location",
                    'lat': location_lat,
                    'lon': location_lon,
                    'address': address
                })
    
    return valid_locations



# Random sampling 

max 500 properties from every suburb in the datasets we have

In [8]:
import pandas as pd

# Open the CSV file ( Chnage this with the full dataset?)
file_path = './data/landing/unfixed_3046_3055_properties.csv'

df = pd.read_csv(file_path)

# Extract postcodes
df.insert(0, 'Postcodes', df['Address'].apply(lambda x: x.split('VIC')[-1].strip()))

# Randomly select max 500 rows from each postcodes and build a dataframe 
df = df.groupby('Postcodes', group_keys=False).apply(lambda x: x.sample(min(len(x), 5)))


FileNotFoundError: [Errno 2] No such file or directory: './data/landing/unfixed_3046_3055_properties.csv'

### Get nearby amenitites for each property 

Using the randomly sampled dataframe and previously built function

Output the coordinates for every found anmentity 

In [51]:
import time
import pandas as pd
from json.decoder import JSONDecodeError

summary_list = []

# Build the filters of amenities
location_types = ["schools", "parks", "supermarkets", "shopping_districts", "train_stations", "hospitals"]

# Create an empty DataFrame with summary columns
summary_df = pd.DataFrame(columns=['Postcodes', 'Address', 'URLS', 'Latitude', 'Longitude', 'Location Type', 'Count', 'Location Name', 'Location Address', 'Location Latitude', 'Location Longitude'])

for index, row in df.iterrows():
    postcodes = row['Postcodes']
    address = row['Address']
    urls = row['URLS']
    lat = row['Latitude']
    lon = row['Longitude']

    summary = {}

    try:
        for location_type in location_types:
            try:
                locations = find_nearby_locations(lat, lon, location_type)
                
                # Check if locations are found and process them
                if locations:
                    valid_locations = process_locations(locations, address)
                    summary[location_type] = {
                        'count': len(locations),  
                        'locations': valid_locations  
                    }
                else:
                    print(f"No locations found for {location_type} at {address}")
            
            # Catch potential JSON errors or other exceptions within find_nearby_locations
            except JSONDecodeError as e:
                print(f"JSON decoding error for {address} and location type {location_type}: {e}")
                continue

            # Add a delay of 1 second after processing each location_type
            time.sleep(1)

    except UnboundLocalError as e:
        print(f"Error processing {address}: {e}")
        continue  # Skip to the next iteration if there's an error

    # Add CBD and its summary as an individual amenity for every property
    summary_df = pd.concat([summary_df, pd.DataFrame([{
        'Postcodes': postcodes,
        'Address': address,
        'URLS': urls,
        'Latitude': lat,
        'Longitude': lon,
        'Location Type': 'CBD',  # Example row
        'Count': '1',
        'Location Name': 'CBD',
        'Location Address': '',
        'Location Latitude': '-37.8124',
        'Location Longitude': '144.9623'
    }])], ignore_index=True)

    # Add the summary of found nearby amenities for every property 
    for location_type, counts in summary.items():
        count = counts['count']
        for loc in counts['locations']:
            summary_df = pd.concat([summary_df, pd.DataFrame([{
                'Postcodes': postcodes,
                'Address': address,
                'URLS': urls,
                'Latitude': lat,
                'Longitude': lon,
                'Location Type': location_type,
                'Count': count,
                'Location Name': loc['name'],
                'Location Address': loc['address'],
                'Location Latitude': loc['lat'],
                'Location Longitude': loc['lon']
            }])], ignore_index=True)

    # Add a delay of 1 second between processing each property
    time.sleep(1)

# Fix
For amentities missing the corrdinates,using the geopy to fix

But their are still some minor errors

Output a fixed properties_stats.csv

In [57]:
import pandas as pd

from geopy.geocoders import Nominatim
import time

# Initialize the geolocator
geolocator = Nominatim(user_agent="geoapi")


# Build another Function to get the latitude and longitude from an address,and handle the errors
def get_lat_lon(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except:
        return None, None

# Iterate through the rows and fill missing lat/lon
for idx, row in summary_df.iterrows():
    if pd.isnull(row['Location Latitude']) or pd.isnull(row['Location Longitude']):
        lat, lon = get_lat_lon(row['Location Address'])
        summary_df.at[idx, 'Location Latitude'] = lat
        summary_df.at[idx, 'Location Longitude'] = lon
        time.sleep(4)  # Sleep to avoid overwhelming the API

# Save the updated data to a new CSV file
summary_df.to_csv('properties_stats.csv', index=False)
